In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'youtube'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

youtube_df = pd.read_sql_query('select * from youtube',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [3]:
youtube_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Rank           5000 non-null   object
 1   Grade          5000 non-null   object
 2   Channel name   5000 non-null   object
 3   Video Uploads  5000 non-null   object
 4   Subscribers    5000 non-null   object
 5   Video views    5000 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 136.8+ KB


 As we will see upon closer investigation, some of the variables that appear as objects are indeed integer variables.

In [4]:
youtube_df.head()

,Rank,Grade,Channel name,Video Uploads,Subscribers,Video views
0,1st,A++,Zee TV,82757,18752951,20869786591
1,2nd,A++,T-Series,12661,61196302,47548839843
2,3rd,A++,Cocomelon - Nursery Rhymes,373,19238251,9793305082
3,4th,A++,SET India,27323,31180559,22675948293
4,5th,A++,WWE,36756,32852346,26273668433


This happened because both Video Uploads and Subscribers contain observations that can't be handled as integers. This includes both missing values and observations stored in the database as --.

To confirm that this is indeed what's happening, let's select the rows where values of the Video Uploads and Subscribers columns are equal to --:

In [5]:
youtube_df[(youtube_df["Video Uploads"].str.strip() == "--") | 
           (youtube_df["Subscribers"].str.strip() == "--")]

,Rank,Grade,Channel name,Video Uploads,Subscribers,Video views
17,18th,A+,Vlad and Nikita,53,--,1428274554
108,109th,A,BIGFUN,373,--,941376171
115,116th,A,Bee Kids Games - Children TV,740,--,414535723
142,143rd,A,ChiChi TV Siêu Nhân,421,--,2600394871
143,144th,A,MusicTalentNow,1487,--,3252752212
...,...,...,...,...,...,...
4941,"4,942nd",B+,GMTV,183,--,127080542
4948,"4,949th",B+,Keivon ToysReview,468,--,481568513
4956,"4,957th",B+,CLICKNEWS,2661,--,139940815
4961,"4,962nd",B+,ONE Championship,905,--,109836654


lets check what's categorical and continuous:

In [6]:
youtube_df.nunique()

Rank             5000
Grade               6
Channel name     4993
Video Uploads    2286
Subscribers      4612
Video views      5000
dtype: int64

grade has 6 - is categorical. channel name stores the name of the youtube channel, so each channel a unique category. then number possibilities this value can take is limited to the number of youtube channels. What about the Rank column? Since this contains the rank of each channel, it can be considered as either an ordinal categorical variable or as an interval continuous variable. The variable type we choose will depend on the task at hand and likely some experimentation on the data to decide which is more helpful

find out the most and least watched channels. 
In this case, we would categorize the channels with respect to their video views into 3 groups: 

1. The most watched channels, where the video views are higher than 1 billion.
2. The moderately watched channels, where the video views are lower than 1 billion and higher than 100 million.
3. The least watched channels, where the video views are lower than 100 million.

Here, we are transforming a *continuous* variable into an *ordinal, categorical* variable. 

Let's create a new feature in our Pandas DataFrame, `views_group`, that does this:

In [7]:
# this method returns group numbers 
# given video views
def categorize_video_views(views_num):
    if views_num >= 1000000000:
        return 1
    elif views_num >= 100000000:
        return 2
    else:
        return 3

# we use Pandas' .apply() method by calling the function above.
youtube_df['views_group'] = youtube_df['Video views'].apply(categorize_video_views)

# let's see how many observations we have in each group
print(youtube_df.groupby("views_group")["Video views"].count())

views_group
1    1399
2    2846
3     755
Name: Video views, dtype: int64


In [10]:
youtube_df.isnull()

,Rank,Grade,Channel name,Video Uploads,Subscribers,Video views,views_group
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
4995,False,False,False,False,False,False,False
4996,False,False,False,False,False,False,False
4997,False,False,False,False,False,False,False
4998,False,False,False,False,False,False,False


In [11]:
for column_name in youtube_df.columns:
    print("Unique values in column {} are: {}".format(column_name, youtube_df[column_name].unique()))

Unique values in column Rank are: ['1st' '2nd' '3rd' ... '4,998th' '4,999th' '5,000th']
Unique values in column Grade are: ['A++ ' 'A+ ' 'A ' '\xa0 ' 'A- ' 'B+ ']
Unique values in column Channel name are: ['Zee TV' 'T-Series' 'Cocomelon - Nursery Rhymes' ... 'Mastersaint'
 'Bruce McIntosh' 'SehatAQUA']
Unique values in column Video Uploads are: ['82757' '12661' '373' ... '1735' '706' '3475']
Unique values in column Subscribers are: ['18752951' '61196302' '19238251' ... '3265735' '32990' '21172']
Unique values in column Video views are: [20869786591 47548839843  9793305082 ...   311758426    14563764
    73312511]
Unique values in column views_group are: [1 2 3]


In [12]:
# print all values that cannot be converted to float
for column_name in ["Video Uploads", "Subscribers"]:
    print("These are the problematic values for the variable: {}".format(column_name))
    for value in youtube_df[column_name]:
        try:
            float(value)
        except:
            print(value)

These are the problematic values for the variable: Video Uploads
--
--
--
--
--
--
These are the problematic values for the variable: Subscribers
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
-- 
--

In [13]:
# replace "--" values to empty strings

youtube_df["Video Uploads"] = youtube_df["Video Uploads"].apply(str.strip).replace("--", np.nan)
youtube_df["Video Uploads"] = pd.to_numeric(youtube_df["Video Uploads"], downcast="float")

youtube_df["Subscribers"] = youtube_df["Subscribers"].apply(str.strip).replace("--", np.nan)
youtube_df["Subscribers"] = pd.to_numeric(youtube_df["Subscribers"], downcast="float")

In [14]:
youtube_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rank           5000 non-null   object 
 1   Grade          5000 non-null   object 
 2   Channel name   5000 non-null   object 
 3   Video Uploads  4994 non-null   float32
 4   Subscribers    4613 non-null   float32
 5   Video views    5000 non-null   int64  
 6   views_group    5000 non-null   int64  
dtypes: float32(2), int64(2), object(3)
memory usage: 175.8+ KB


In [15]:
youtube_df.isnull()

,Rank,Grade,Channel name,Video Uploads,Subscribers,Video views,views_group
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
4995,False,False,False,False,False,False,False
4996,False,False,False,False,False,False,False
4997,False,False,False,False,False,False,False
4998,False,False,False,False,False,False,False


We can see some True values (check row 17). So far, so good! To get a sense of the magnitude of missing values, let's calculate the percentages of the missing values in all columns

In [16]:
youtube_df.isnull().sum()*100/youtube_df.isnull().count()

Rank             0.00
Grade            0.00
Channel name     0.00
Video Uploads    0.12
Subscribers      7.74
Video views      0.00
views_group      0.00
dtype: float64

In [17]:
youtube_df.Grade.value_counts()

B+      2956
A-      1024
A        963
A+        41
A++       10
           6
Name: Grade, dtype: int64

In [18]:
youtube_df.Grade = youtube_df.Grade.apply(str.strip).replace("", np.nan)

youtube_df.Grade.unique()

array(['A++', 'A+', 'A', nan, 'A-', 'B+'], dtype=object)

filling in missing values of a variable with the mean, the median or the mode of that variable. This is called missing value imputation

In [19]:
youtube_df["Video Uploads"].fillna(youtube_df["Video Uploads"].mean(), inplace=True)
youtube_df["Subscribers"].fillna(youtube_df["Subscribers"].mean(), inplace=True)

# Be careful with inplace=True. It works in this case, but sometimes it is safer to use this:
# youtube_df["Video Uploads"] = youtube_df["Video Uploads"].fillna(youtube_df["Video Uploads"].mean())
# youtube_df["Subscribers"] = youtube_df["Subscribers"].fillna(youtube_df["Subscribers"].mean())
# (See the Assignment example solution for more detail on this issue)

In [20]:
youtube_df.isnull().sum()*100/youtube_df.count()

Rank             0.000000
Grade            0.120144
Channel name     0.000000
Video Uploads    0.000000
Subscribers      0.000000
Video views      0.000000
views_group      0.000000
dtype: float64

In [21]:
# fill in missing values in Category with the the value of the next highest-ranked channel's category

grade_list = youtube_df.Grade

for i in range(0, len(youtube_df.Grade)):
    if pd.isnull(youtube_df.Grade[i]):
        youtube_df.Grade[i] = youtube_df.Grade[i-1]
        
youtube_df["Grade"].unique()

array(['A++', 'A+', 'A', 'A-', 'B+'], dtype=object)